<a href="https://colab.research.google.com/github/YLysov0017/Pproject/blob/master/ML_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Задание по практике 3

##Обработка пропущенных значений

Обработайте пропущенные значения:


* Удалите пропущенные значения
* Заполните при помощи медианы, среднего для числовых значений
* Заполните при помощи моды для категориальных значений
* Интерполируйте
Выведите статистку после каждого метода и объясните


In [1]:
!pip install scikit-learn

In [33]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/crimes_against_women_2001-2014.csv')
df["Dowry Deaths"].describe()


count    10677.000000
mean        20.181699
std         98.276531
min          0.000000
25%          1.000000
50%          5.000000
75%         16.000000
max       2469.000000
Name: Dowry Deaths, dtype: float64

In [7]:
def create_holes(dataframe, column_name, hole_percentage):
    """
    Создает "дыры" в указанной колонке заданного DataFrame.

    Parameters:
        dataframe (pd.DataFrame): Исходный DataFrame.
        column_name (str): Имя колонки, в которой нужно создать "дыры".
        hole_percentage (float): Процент дырявости, который указывает, какую часть данных следует заменить на NaN.

    Returns:
        pd.DataFrame: DataFrame с "дырами" в указанной колонке.
    """
    if hole_percentage < 0 or hole_percentage > 100:
        raise ValueError("Процент дырявости должен быть в диапазоне от 0 до 100.")

    dataframe_with_holes = dataframe.copy()
    num_holes = int(len(dataframe) * (hole_percentage / 100))
    hole_indices = np.random.choice(len(dataframe), num_holes, replace=False)
    dataframe_with_holes.loc[hole_indices, column_name] = np.nan

    return dataframe_with_holes

In [40]:
df_dirty = create_holes(df, "Dowry Deaths", 20)
print(df_dirty["Dowry Deaths"].isnull().sum())
df_dirty["Dowry Deaths"].describe()


2135


count    8542.000000
mean       19.658628
std        94.908798
min         0.000000
25%         0.000000
50%         5.000000
75%        16.000000
max      2469.000000
Name: Dowry Deaths, dtype: float64

In [35]:
#Удаление
df_cleaned = df_dirty.dropna()
print(df_cleaned["Dowry Deaths"].isnull().sum())

df_cleaned["Dowry Deaths"].describe()


0


count    8542.000000
mean       20.807656
std       103.273478
min         0.000000
25%         1.000000
50%         5.000000
75%        16.000000
max      2469.000000
Name: Dowry Deaths, dtype: float64

In [36]:
#Заполнение при помощи медианы
df_filled_median = df_dirty.fillna(df_dirty.median())

df_filled_median["Dowry Deaths"].describe()



<ipython-input-36-bc9fbef355ad>:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_filled_median = df_dirty.fillna(df_dirty.median())


count    10677.000000
mean        17.646717
std         92.587839
min          0.000000
25%          1.000000
50%          5.000000
75%         12.000000
max       2469.000000
Name: Dowry Deaths, dtype: float64

In [37]:
#Заполнение при помощи среднего
df_filled_mean = df_dirty.fillna(df_dirty.mean())

df_filled_mean["Dowry Deaths"].describe()


<ipython-input-37-2a9c3baf1a3a>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_filled_mean = df_dirty.fillna(df_dirty.mean())


count    10677.000000
mean        20.807656
std         92.371688
min          0.000000
25%          1.000000
50%         10.000000
75%         20.807656
max       2469.000000
Name: Dowry Deaths, dtype: float64

In [38]:
#Заполнение при помощи моды
df_filled_mode = df_dirty.fillna(df_dirty["Assault on women with intent to outrage her modesty"].mode()[0])

df_filled_mode["Dowry Deaths"].describe()


count    10677.000000
mean        16.646905
std         92.745883
min          0.000000
25%          0.000000
50%          3.000000
75%         12.000000
max       2469.000000
Name: Dowry Deaths, dtype: float64

In [39]:
#Заполнение при помощи интерполяции
df_interpolated = df_dirty.interpolate()

df_dirty["Dowry Deaths"].describe()


count    8542.000000
mean       20.807656
std       103.273478
min         0.000000
25%         1.000000
50%         5.000000
75%        16.000000
max      2469.000000
Name: Dowry Deaths, dtype: float64

**Объясните**

Все методы изменили стандартное отклонение в ту или иную сторону. Заполнение модой уменьшило число свыше 75 квантиля, медиана его увеличила

##Поиск выбросов и аномалий

**Те, кто выполняет задания со * делают поиск выбросов с помощью написанного ранее класса**

Найти выбросы по числовой переменной (использовать метод основанный на среднем и на межквартильном размахе). Объясните результат

Вывести статистику с выбросами и без. Сравните и объясните результат


In [71]:
#Методом среднего


mean_data = df["Dowry Deaths"].mean()
maximum = mean_data * 2
res = []
for item in df["Dowry Deaths"]:
  if abs(item - mean_data) < maximum:
    res.append(item)

df_mean = df[abs((df["Dowry Deaths"] - mean_data) < maximum)]
df_mean["Dowry Deaths"].describe()


count    10286.000000
mean         9.533444
std         11.916457
min          0.000000
25%          0.000000
50%          5.000000
75%         14.000000
max         60.000000
Name: Dowry Deaths, dtype: float64

In [67]:
#При помощи межквартильного размаха
Q1 =  df["Dowry Deaths"].quantile(0.25)
Q3 = df["Dowry Deaths"].quantile(0.75)
IQR = Q3 - Q1
df_IQR = df[(df["Dowry Deaths"] > (Q1-1.5*IQR )) & (df["Dowry Deaths"] < (Q3+1.5*IQR))]
df_IQR["Dowry Deaths"].describe()

count    9876.000000
mean        7.995038
std         9.330764
min         0.000000
25%         0.000000
50%         4.000000
75%        13.000000
max        38.000000
Name: Dowry Deaths, dtype: float64

In [ ]:
#Статистика и объяснение

Отсечение по межквартильному размаху оставляет меньше значений и сильнее срезает максимум

**Объяснения**

##Дубликаты

Проверьте датафрейм на дубликаты и удалите их, если дублей нет - создайте.

In [47]:
#Проверка на дубли
dupls = df.duplicated()

count = 0
for i in dupls:
  if (i): count +=1

count


10677


In [43]:
df_with_dupls = pd.concat((df, df.loc[0:100]))
dupls = df_with_dupls.duplicated()
count = 0
for i in dupls:
  if (i): count +=1

count

101

In [48]:
#Удаление дублей
df_no_duplicates = df_with_dupls.drop_duplicates()
df_no_duplicates

count = 0
for i in dupls:
  if (i): count +=1

count

0

## Преобразование признаков

Масштабируйте числовые признаки через MinMaxScaler, StandardScaler

In [49]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Creating instances
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

num_features = df.select_dtypes(include=['int']).columns
scaler_minmax = MinMaxScaler()
min_max_scaler = df.copy()
min_max_scaler[num_features] = scaler_minmax.fit_transform(df[num_features])
min_max_scaler["Dowry Deaths"].unique()


array([6.48035642e-03, 2.83515593e-03, 5.67031187e-03, 6.88537870e-03,
       4.86026731e-03, 0.00000000e+00, 9.72053463e-03, 2.51113811e-02,
       4.05022276e-03, 5.26528959e-03, 1.05305792e-02, 1.25556906e-02,
       7.69542325e-03, 2.02511138e-03, 1.49858242e-02, 4.05022276e-04,
       2.43013366e-03, 8.10044552e-04, 1.21506683e-03, 2.10611584e-02,
       6.07533414e-03, 1.70109356e-01, 3.64520049e-03, 1.62008910e-03,
       2.38963143e-02, 4.45524504e-03, 1.33657351e-02, 8.50546780e-03,
       3.24017821e-03, 8.10044552e-03, 9.31551235e-03, 8.91049008e-03,
       1.01255569e-02, 2.79465371e-02, 1.90360470e-02, 3.64520049e-02,
       1.45808019e-02, 2.71364925e-02, 1.29607128e-02, 3.47914135e-01,
       2.83515593e-02, 7.29040097e-03, 1.15431349e-01, 1.21506683e-02,
       8.78898339e-02, 1.62008910e-02, 1.13406237e-02, 8.91049008e-02,
       1.09356015e-02, 1.37707574e-02, 1.66059133e-02, 1.17456460e-02,
       2.46658566e-01, 1.41757797e-02, 1.53908465e-02, 1.24746861e-01,
      

In [51]:
standard_scaler = StandardScaler()
st_scaler = df.copy()
st_scaler[num_features] = standard_scaler.fit_transform(df[num_features])
st_scaler["Dowry Deaths"].unique()

array([-4.25523245e-02, -1.34134938e-01, -6.29040164e-02, -3.23764786e-02,
       -8.32557082e-02, -2.05365859e-01,  3.88544429e-02,  4.25536588e-01,
       -1.03607400e-01, -7.30798623e-02,  5.92061347e-02,  1.10085364e-01,
       -1.20247867e-02, -1.54486630e-01,  1.71140440e-01, -1.95190013e-01,
       -1.44310784e-01, -1.85014167e-01, -1.74838321e-01,  3.23778129e-01,
       -5.27281704e-02,  4.06848943e+00, -1.13783246e-01, -1.64662476e-01,
        3.95009050e-01, -9.34315541e-02,  1.30437056e-01,  8.32690511e-03,
       -1.23959092e-01, -1.84894082e-03,  2.86785970e-02,  1.85027510e-02,
        4.90302888e-02,  4.96767509e-01,  2.72898899e-01,  7.10460274e-01,
        1.60964594e-01,  4.76415818e-01,  1.20261210e-01,  8.53568579e+00,
        5.06943355e-01, -2.22006327e-02,  2.69475023e+00,  9.99095184e-02,
        2.00279271e+00,  2.01667978e-01,  7.95578266e-02,  2.03332024e+00,
        6.93819806e-02,  1.40612902e-01,  2.11843824e-01,  8.97336725e-02,
        5.99172431e+00,  

Масштабируйте категориальные признаки через OneHotEncoder

In [57]:
from sklearn.preprocessing import OneHotEncoder
df["STATE/UT "] = df["STATE/UT"].astype("category")
df["DISTRICT"] = df["DISTRICT"].astype("category")


# Create an instance of OneHotEncoder
encoder = OneHotEncoder()
cat_features = df.select_dtypes(include=['category']).columns
x = encoder.fit_transform(df[["STATE/UT"]]).toarray()
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

##Задание для гениев*


Реализовать расчет корреляции Спирмана

In [ ]:
#Функция по расчету тут

In [ ]:
#Пример использования тут и объяснение принципа работы

###Пример реализации

In [ ]:
import pandas as pd
import numpy as np


def corr_pearson(column_1, column_2):
  cov = np.cov(column_1, column_2)
  return cov/(column_1.std() * column_2.std())

In [ ]:
data = {
    'Рост': [170, 155, 180, 162, 175],
    'Вес': [65, 50, 75, 55, 70]
}
df = pd.DataFrame(data)
df

,Рост,Вес
0,170,65
1,155,50
2,180,75
3,162,55
4,175,70


In [ ]:
corr_pearson(df["Рост"], df["Вес"])

array([[0.96593129, 0.99674864],
       [0.99674864, 1.03527033]])